# Inteligência Artificial e Aprendizado de Máquina

**Prof. Dr. Francisco de Assis Boldt**  
**Pós-graduação em Desenvolvimento de Aplicações Inteligentes**  
**Inteligência Artificial e Aprendizado de Máquina — Atividade 01 - Regressão**  

**Nome:** Otávio Lube dos Santos  
**Matrícula:** 20231DEVAI0157

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/titanic/train.csv")
train.head()

In [ ]:
test = pd.read_csv("/kaggle/input/titanic/test.csv")
test.head()

In [ ]:
# Preenchendo valores ausentes
train['Age'].fillna(train['Age'].median(), inplace=True)
test['Age'].fillna(test['Age'].median(), inplace=True)
train['Embarked'].fillna(train['Embarked'].mode()[0], inplace=True)
test['Fare'].fillna(test['Fare'].median(), inplace=True)

In [ ]:
# Convertendo variáveis categóricas em numéricas
train['Sex'] = train['Sex'].map({'male': 0, 'female': 1})
test['Sex'] = test['Sex'].map({'male': 0, 'female': 1})
train['Embarked'] = train['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})
test['Embarked'] = test['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})

In [ ]:
# Criando novas features
train['FamilySize'] = train['SibSp'] + train['Parch'] + 1
test['FamilySize'] = test['SibSp'] + test['Parch'] + 1
train['IsAlone'] = (train['FamilySize'] == 1).astype(int)
test['IsAlone'] = (test['FamilySize'] == 1).astype(int)

In [ ]:
# Selecionando features
features = ['Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'FamilySize', 'IsAlone']
X = train[features].copy()
y = train['Survived']
X_test = test[features].copy()

In [ ]:
# Garantindo que não existam valores ausentes nos datasets
X.fillna(X.median(), inplace=True)
X_test.fillna(X_test.median(), inplace=True)

In [ ]:
# Dividindo os dados em treino e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.fillna(X_train.median(), inplace=True)
X_val.fillna(X_val.median(), inplace=True)

In [ ]:
# Construindo modelos base
log_clf = LogisticRegression(max_iter=1000, random_state=42)
rnd_clf = RandomForestClassifier(random_state=42, n_estimators=100, max_depth=5)
svm_clf = SVC(probability=True, random_state=42)

In [ ]:
# Construindo o StackingClassifier
stacking_clf = StackingClassifier(
    estimators=[
        ('lr', log_clf),
        ('rf', rnd_clf),
        ('svc', svm_clf)
    ],
    final_estimator=LogisticRegression(random_state=42)
)

In [ ]:
# Treinando o StackingClassifier
stacking_clf.fit(X_train, y_train)

In [ ]:
# Avaliando no conjunto de validação
y_pred = stacking_clf.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f"Acurácia no conjunto de validação: {accuracy:.4f}")

In [ ]:
# Previsões no conjunto de teste
test_predictions = stacking_clf.predict(X_test)

In [ ]:
# Criando o arquivo de submissão
submission = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": test_predictions
})

In [ ]:
submission.to_csv("submission.csv", index=False)
print("Submissão criada: submission.csv")